In [1]:
!nvidia-smi

Mon Oct 21 08:40:55 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4000               On  |   00000000:41:00.0 Off |                  Off |
| 41%   31C    P8             11W /  140W |       2MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [24]:
# !pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"
# !pip install scikit-learn
!pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('dataset/parallel-corpus.xlsx')

# Display the DataFrame
df.head()

,SENTENCES,MEANING,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,How can I communicate with my parents?,میں اپنے والدین سے کیسے بات کروں ؟,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,How can I make friends?’,میں دوست کیسے بنائوں ؟,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Why do I get so sad?’,میں اتنا اداس کیوں ہوں؟.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"If you’ve asked yourself such questions, you’r...",اگر آپ نے اپنے آپ سے ایسے سوالات کیے ہیں، تو آ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Depending on where you’ve turned for guidance,...",اس بات پر منحصر ہے کہ آپ رہنمائی کے لیے کہاں ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
len(df)

30164

In [10]:
len(df) * 0.2

6032.8

In [23]:
from sklearn.model_selection import train_test_split

# Split the data into training+validation and test sets first (e.g., 80% train+validation, 20% test)
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Now split the training+validation set into training and validation sets (e.g., 75% train, 25% validation)
val, test = train_test_split(test, test_size=0.10, random_state=42)  # 0.25 * 0.8 = 0.2 validation

# Now, 'train' is 60%, 'val' is 20%, and 'test' is 20%
print(f"Training set size: {len(train)}")
print(f"Validation set size: {len(val)}")
print(f"Test set size: {len(test)}")

Training set size: 24131
Validation set size: 5429
Test set size: 604


# Gemma 2 2b

In [16]:
from unsloth import FastLanguageModel

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/gemma-7b-it-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-it-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.10.3: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.724 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [36]:
import time

start_time = time.time()


# example
eng_sentence = "How are you?"
# urdu_sentence = train_data.translation[2].get("ur")

prompt = """
<start_of_turn>user
Translate the sentence into urdu and only return the translation:
 
This is a spoon<end_of_turn>
<start_of_turn>model
"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(prompt, return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
gen_tokens = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

outs = tokenizer.batch_decode(gen_tokens[:, inputs.input_ids.shape[1]:])[0]
response_only = outs.replace(tokenizer.eos_token, "").replace("\n<end_of_turn>", "")
print(response_only)

print("--- %s seconds ---" % (time.time() - start_time))

<bos>
<start_of_turn>user
Translate the sentence into urdu and only return the translation:
 
This is a spoon<end_of_turn>
<start_of_turn>model
 یہ ایک چنگا ہے۔ 
<end_of_turn>
 یہ ایک چنگا ہے۔ 
--- 0.41672229766845703 seconds ---


In [37]:
response_only

' یہ ایک چنگا ہے۔ '

# English to urdu on pretrained Gemma 2 2b baseline

In [38]:
from tqdm import tqdm
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Define the column you want to process and modify
for index, row in tqdm(test.iterrows()):
    input_sentence = row["SENTENCES "]
    
    prompt = f"""
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    {input_sentence}<end_of_turn>
    <start_of_turn>model
    """

    # alpaca_prompt = You MUST copy from above!

    inputs = tokenizer(prompt, return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    gen_tokens = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128);

    outs = tokenizer.batch_decode(gen_tokens[:, inputs.input_ids.shape[1]:])[0]
    response_only = outs.replace(tokenizer.eos_token, "").replace("\n<end_of_turn>", "")
    
    test.at[index, 'en_ur_2b_bl'] = response_only

test.head()

0it [00:00, ?it/s]

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    Ambiance was amazing and so the staff. Quality of food was also good but the rates are quit high. Their beef mince burger was not up to the mark.
Overall a nice place to visit.<end_of_turn>
    <start_of_turn>model
    فضاء بہت خوب تھا اورservants بھی بہت خوب تھے۔  غذای کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی کی


1it [00:04,  5.00s/it]

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    You have to promise never to tell anyone what I'm about to tell you.<end_of_turn>
    <start_of_turn>model
    آپ کو یہ کہنا پڑے گا کہ میں یہ کہنے والا نہیں ہے، کسی کو نہیں بتا سکتا۔ 
<end_of_turn>


2it [00:06,  2.80s/it]

...

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    ”

The owner did not wait for a second invitation<end_of_turn>
    <start_of_turn>model
    مالک نے دوسری دعوت کے لیے انتظار نہیں کیا۔ 
<end_of_turn>


603it [14:28,  1.40s/it]

<bos>
    <start_of_turn>user
    Translate the sentence into urdu and only return the translation:

    Four boys are making their way along the street.<end_of_turn>
    <start_of_turn>model
    چهار хлопکوں کو عمارت پر چڑھنے کی جارہی ہے۔ 
<end_of_turn>


604it [14:29,  1.44s/it]


,SENTENCES,MEANING,en_ur_2b_bl
21101,Ambiance was amazing and so the staff. Quality...,ماحول حیرت انگیز تھا اور عملہ بھی۔ کھانے کا مع...,فضاء بہت خوب تھا اورservants بھی بہت خوب تھے۔ ...
1889,You have to promise never to tell anyone what ...,تم نے وعدہ کرنا ہے کہ جو میں تمہیں بتانے لگا ہ...,آپ کو یہ کہنا پڑے گا کہ میں یہ کہنے والا نہیں ...
8971,Good place with average food quality,کھانے کے اوسط معیار کے ساتھ اچھی جگہ,خوبہ مکمل غذای میان
15662,And he lifted his father and mother upon the t...,(شہر میں داخل ہونے کے بعد) اس نے اپنے والدین ک...,وہ اپنے پڑھیاں اور مہم کی چھڑی پر چڑھ لیا، اور...
11374,What a great place for family safe zone,فیملی سیف زون کے لیے کتنی اچھی جگہ ہے۔,یہ ایک عائلی محفوظ گھڑی ہے


In [39]:
# Drop all columns with names that start with 'Unnamed'
# test = test.loc[:, ~test.columns.str.contains('^Unnamed')]

test.head()

,SENTENCES,MEANING,en_ur_2b_bl
21101,Ambiance was amazing and so the staff. Quality...,ماحول حیرت انگیز تھا اور عملہ بھی۔ کھانے کا مع...,فضاء بہت خوب تھا اورservants بھی بہت خوب تھے۔ ...
1889,You have to promise never to tell anyone what ...,تم نے وعدہ کرنا ہے کہ جو میں تمہیں بتانے لگا ہ...,آپ کو یہ کہنا پڑے گا کہ میں یہ کہنے والا نہیں ...
8971,Good place with average food quality,کھانے کے اوسط معیار کے ساتھ اچھی جگہ,خوبہ مکمل غذای میان
15662,And he lifted his father and mother upon the t...,(شہر میں داخل ہونے کے بعد) اس نے اپنے والدین ک...,وہ اپنے پڑھیاں اور مہم کی چھڑی پر چڑھ لیا، اور...
11374,What a great place for family safe zone,فیملی سیف زون کے لیے کتنی اچھی جگہ ہے۔,یہ ایک عائلی محفوظ گھڑی ہے


In [40]:
test.to_csv('test_data.csv', index=False)

# Urdu to English on pretrained Gemma 2 2b baseline

In [41]:
# Assuming 'test' is your DataFrame
column_names = test.columns

# Display the column names
print(column_names)

Index(['SENTENCES ', 'MEANING', 'en_ur_2b_bl'], dtype='object')


In [42]:
from tqdm import tqdm
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Define the column you want to process and modify
for index, row in tqdm(test.iterrows()):
    input_sentence = row["MEANING"]
    
    prompt = f"""
    <start_of_turn>user
    Translate the sentence into english and only return the translation:

    {input_sentence}<end_of_turn>
    <start_of_turn>model
    """

    # alpaca_prompt = You MUST copy from above!

    inputs = tokenizer(prompt, return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)
    gen_tokens = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128);

    outs = tokenizer.batch_decode(gen_tokens[:, inputs.input_ids.shape[1]:])[0]
    response_only = outs.replace(tokenizer.eos_token, "").replace("\n<end_of_turn>", "")
    
    test.at[index, 'ur_en_2b_bl'] = response_only

test.head()

0it [00:00, ?it/s]

<bos>
    <start_of_turn>user
    Translate the sentence into english and only return the translation:

    ماحول حیرت انگیز تھا اور عملہ بھی۔ کھانے کا معیار بھی اچھا تھا لیکن قیمتیں زیادہ ہیں۔ ان کا بیف کیما برگر نشان تک نہیں تھا۔ مجموعی طور پر دیکھنے کے لیے ایک اچھی جگہ۔<end_of_turn>
    <start_of_turn>model
    It was quite surprising, the atmosphere and the food was good, but the prices are high. Their beef kofta burger wasn't up to the mark. Overall, a good place to visit. 
<end_of_turn>


1it [00:01,  1.77s/it]

<bos>
    <start_of_turn>user
    Translate the sentence into english and only return the translation:

    تم نے وعدہ کرنا ہے کہ جو میں تمہیں بتانے لگا ہوں، وہ کسی اور کو کبھی نہیں پتانا ہے۔<end_of_turn>
    <start_of_turn>model
    I promise that whatever I tell you, I will never tell anyone else. 
<end_of_turn>


2it [00:02,  1.15s/it]

...

<bos>
    <start_of_turn>user
    Translate the sentence into english and only return the translation:

    مالک نے دوسری دعوت کا انتظار نہیں کیا۔<end_of_turn>
    <start_of_turn>model
    The owner did not wait for a second invitation. 
<end_of_turn>


603it [07:31,  1.28it/s]

<bos>
    <start_of_turn>user
    Translate the sentence into english and only return the translation:

    چار لڑکے سڑک پر جا رہے ہیں۔<end_of_turn>
    <start_of_turn>model
    Four boys are walking on the street. 
<end_of_turn>


604it [07:31,  1.34it/s]


,SENTENCES,MEANING,en_ur_2b_bl,ur_en_2b_bl
21101,Ambiance was amazing and so the staff. Quality...,ماحول حیرت انگیز تھا اور عملہ بھی۔ کھانے کا مع...,فضاء بہت خوب تھا اورservants بھی بہت خوب تھے۔ ...,"It was quite surprising, the atmosphere and th..."
1889,You have to promise never to tell anyone what ...,تم نے وعدہ کرنا ہے کہ جو میں تمہیں بتانے لگا ہ...,آپ کو یہ کہنا پڑے گا کہ میں یہ کہنے والا نہیں ...,"I promise that whatever I tell you, I will nev..."
8971,Good place with average food quality,کھانے کے اوسط معیار کے ساتھ اچھی جگہ,خوبہ مکمل غذای میان,Good place with average food.
15662,And he lifted his father and mother upon the t...,(شہر میں داخل ہونے کے بعد) اس نے اپنے والدین ک...,وہ اپنے پڑھیاں اور مہم کی چھڑی پر چڑھ لیا، اور...,"After entering the city, he picked up his pare..."
11374,What a great place for family safe zone,فیملی سیف زون کے لیے کتنی اچھی جگہ ہے۔,یہ ایک عائلی محفوظ گھڑی ہے,"""This is a good place for the family."""


In [43]:
test.to_csv('test_data.csv', index=False)